In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import all files from tDCS FOG together into a single data frame.

In [ ]:
# Import all files of train tdcsfo
from pathlib import Path
path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/'
all_files= Path(path).glob('*.csv')

dfs = list()
for f in all_files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['Id'] = f.stem
    dfs.append(data)

train_tdfog = pd.concat(dfs, ignore_index=True)





Time An integer timestep. Series from the tdcsfog dataset are recorded at 128Hz (128 timesteps per second), while series from the defog and daily series are recorded at 100Hz (100 timesteps per second).

In [ ]:
train_tdfog

* Accelaration from sensor (lower back):
* AccV = Vertial (Up/Down)
* AccML = Mid (Side-Side)
* AccAP = Anterposterior (Back-Foward)

### EDA

In [ ]:
# Looking for null values

nulls = train_tdfog.isnull().sum().to_frame("nulls")
nulls.sort_values("nulls", inplace = True, ascending = False)
nulls

In [ ]:
train_tdfog.describe()

#### Looking for numerical and categorical features

In [ ]:
numerical_features = train_tdfog.select_dtypes(include = [np.number])
numerical_features.columns

In [ ]:
categorical_features = train_tdfog.select_dtypes(include= [object])
categorical_features.columns

In [ ]:
print(train_tdfog['Turn'].unique())
print(train_tdfog['Walking'].unique())

### Combine and explore train defog files

In [ ]:
# Import all files of train tdcsfo
from pathlib import Path
path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog/'
all_files= Path(path).glob('*.csv')

dfs = list()
for f in all_files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['Id'] = f.stem
    dfs.append(data)

train_defog = pd.concat(dfs, ignore_index=True)

train_defog


Time An integer timestep. Series from the tdcsfog dataset are recorded at 128Hz (128 timesteps per second), while series from the defog and daily series are recorded at 100Hz (100 timesteps per second).

In [ ]:
def get_data(f):
    df = pd.read_csv(f)
    df['Id'] = f.split('/')[-1].split('.')[0]
    df['data_type'] = f.split('/')[-2]
    return df

In [ ]:
# Frequency defined in Hz
def frequency_diff(data, frequency):
    
    if frequency == 100:
        data['Time'] /=100
        data['Time'] = round(data['Time'], 3)
        
    elif frequency == 128:
        data['Time'] = round(data['Time']/ 128,)
        
    return data

AccV, AccML, and AccAP Acceleration from a lower-back sensor on three axes: V - vertical, ML - mediolateral, AP - anteroposterior. Data is in units of m/s^2 for tdcsfog/ and g for defog/ and notype/.

In [ ]:
def convert_g(data):
    data['AccV'] *= 9.80665
    data['AccML'] *= 9.80665
    data['AccAP'] *= 9.80665
    return data

### Explore tdcsfog metadata

In [ ]:
fdir = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'
tdfog_meta = pd.read_csv(os.path.join(fdir, 'tdcsfog_metadata.csv'))

tdfog_meta

In [ ]:
tdfog_meta.describe(include = 'all')

In [ ]:
# Nulls
percentage = tdfog_meta.isnull().mean()*100
percentage = percentage.to_frame("nulls")
percentage.sort_values("nulls", inplace = True, ascending = False)
percentage

### Explore defog metadata

In [ ]:
defog_meta = pd.read_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv")
defog_meta

### Explore Tasks Data

In [ ]:
tasks = pd.read_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tasks.csv")

tasks

### Explore Subject Data

In [ ]:
subject = pd.read_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/subjects.csv")
subject

### Explore Events Data

In [ ]:
events = pd.read_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/events.csv")
events

### Combine tdfog meta and defog meta into a single Data Fram

In [ ]:
# Create a new column which identifies which files are from which data


meta_data = pd.concat([defog_meta,tdfog_meta])

meta_data

In [ ]:
meta_data.replace('on',1,inplace=True)
meta_data.replace('off',0,inplace=True)
meta_data

### Merge metadata with subjects

In [ ]:
meta_merge = meta_data.merge(subject,how='left',on='Subject').copy()
meta_merge.fillna(0,inplace = True)
meta_merge

### Combine train defog and tdcsfog files into 1 Data Frame

#### Apply axuiliary functions

In [ ]:
new_train_defog = glob.glob('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog/**')
new_train_tdfog = glob.glob('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/**')
df_train_defog = pd.concat([get_data(f) for f in new_train_defog])
df_train_tdcsfog = pd.concat([get_data(f) for f in new_train_tdfog])


df_train_defog

In [ ]:
df_train_tdcsfog

In [ ]:
df_train_defog

In [ ]:
df_train_defog = convert_g(df_train_defog)

In [ ]:
final_train = pd.concat([df_train_defog, df_train_tdcsfog])

final_train

In [ ]:
#final_train.dropna(axis = 0, how = 'any', inplace =True)
#final_train

In [ ]:
print(final_train.dtypes)


In [ ]:
final_train

In [ ]:
final_train['Valid'].unique()

## Combine train data and meta data

In [ ]:
train_results = final_train.merge(meta_merge, how ='left', on= 'Id').copy()

train_results

In [ ]:
train_results.dropna(axis = 0, how = 'any', inplace =True)
train_results

In [ ]:
train_results = train_results[train_results.Valid != 0]
train_results = train_results[train_results.Task != 0]
train_results

In [ ]:
train_results['data_type'] = np.where(train_results['data_type'] != 'defog', 0, 1)


In [ ]:
train_results.drop(['Age', 'YearsSinceDx'], axis=1)

In [ ]:
X = train_results.iloc[:, 1:4] 


y1 = train_results['StartHesitation']  # target variable for StartHesitation
y2 = train_results['Turn']  # target variable for Turn
y3 = train_results['Walking']  # target variable for Walking

print(X,y1,y2,y3)

In [ ]:
# Find the positions of y1 where it equals 0.
y1_zeros = np.where(y1 == 0)[0]
y1_ones = np.where(y1 == 1)[0]

# Choose the same number of samples with y1 == 1 as there are with y1 == 0.
num1_ones = (y1 == 1).sum()
np.random.seed(42)
y1_zeros = np.random.choice(np.where(y1 == 0)[0], size = num1_ones, replace = False)

# Combine the positions of y1 == 0 and y1 == 1.
y1_balanced_idxs = np.sort(np.concatenate([y1_zeros, y1_ones]))

# Use the balanced indices to get the corresponding rows of X and y1.
X1_balanced = X.iloc[y1_balanced_idxs, :]
y1_balanced = y1.iloc[y1_balanced_idxs]

In [ ]:
# Find the positions of y2 where it equals 0.
y2_zeros = np.where(y2 == 0)[0]
y2_ones = np.where(y2 == 1)[0]

# Choose the same number of samples with y2 == 1 as there are with y2 == 0.
num2_ones = (y2 == 1).sum()
np.random.seed(42)
y2_zeros = np.random.choice(np.where(y2 == 0)[0], size = num2_ones, replace = False)

# Combine the positions of y2 == 0 and y2 == 1.
y2_balanced_idxs = np.sort(np.concatenate([y2_zeros, y2_ones]))

# Use the balanced indices to get the corresponding rows of X and y1.
X2_balanced = X.iloc[y2_balanced_idxs, :]
y2_balanced = y2.iloc[y2_balanced_idxs]

In [ ]:
# Find the positions of y3 where it equals 0.
y3_zeros = np.where(y3 == 0)[0]
y3_ones = np.where(y3 == 1)[0]

# Choose the same number of samples with y3 == 1 as there are with y3 == 0.
num3_ones = (y3 == 1).sum()
np.random.seed(42)
y3_zeros = np.random.choice(np.where(y3 == 0)[0], size = num3_ones, replace = False)

# Combine the positions of y3 == 0 and y3 == 1.
y3_balanced_idxs = np.sort(np.concatenate([y3_zeros, y3_ones]))

# Use the balanced indices to get the corresponding rows of X and y3.
X3_balanced = X.iloc[y3_balanced_idxs, :]
y3_balanced = y3.iloc[y3_balanced_idxs]

### Alg

In [ ]:

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1_balanced, y1_balanced, test_size=.2, random_state=42)
print(X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2_balanced, y2_balanced, test_size=.2, random_state=42)
print(X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3_balanced, y3_balanced, test_size=.2, random_state=42)
print(X3_train.shape, X3_test.shape, y3_train.shape, y3_test.shape)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create three separate logistic regression models.
gmodel1 = GradientBoostingClassifier(verbose = 1)
gmodel2 = GradientBoostingClassifier(verbose = 1)
gmodel3 = GradientBoostingClassifier(verbose = 1)

# Train the models on the training data.
gmodel1.fit(X1_train, y1_train)
gmodel2.fit(X2_train, y2_train)
gmodel3.fit(X3_train, y3_train)

In [ ]:
# Get the predictions for the three models on the test data.
gy1_pred = gmodel1.predict(X1_test)
gy2_pred = gmodel2.predict(X2_test)
gy3_pred = gmodel3.predict(X3_test)

# Create a classification report for each model.
print('Classification Report for StartHesitation:')
print(classification_report(y1_test, gy1_pred))

print('Classification Report for Turn:')
print(classification_report(y2_test, gy2_pred))

print('Classification Report for Walking:')
print(classification_report(y3_test, gy3_pred))

from sklearn.metrics import average_precision_score

print(average_precision_score(y1_test, gy1_pred))
print(average_precision_score(y2_test, gy2_pred))
print(average_precision_score(y3_test, gy3_pred))


## Data Test

In [ ]:
# Set the directory path to the folder containing the CSV files.
tdcsfog_test_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog'

# Initialize an empty list to store the dataframes.
tdcsfog_test_list = []

# Loop through each file in the directory and read it into a dataframe.
for file_name in os.listdir(tdcsfog_test_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(tdcsfog_test_path, file_name)
        file = pd.read_csv(file_path)
        file['Id'] = file_name[:-4] + '_' + file['Time'].apply(str)
        tdcsfog_test_list.append(file)

# Concatenate the dataframes vertically using pd.concat().
tdcsfog_test = pd.concat(tdcsfog_test_list, axis = 0)

# Show the concatenated dataframe.
tdcsfog_test

In [ ]:
# Set the directory path to the folder containing the CSV files.
defog_test_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog'

# Initialize an empty list to store the dataframes.
defog_test_list = []

# Loop through each file in the directory and read it into a dataframe.
for file_name in os.listdir(defog_test_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(defog_test_path, file_name)
        file = pd.read_csv(file_path)
        file['Id'] = file_name[:-4] + '_' + file['Time'].apply(str)
        defog_test_list.append(file)

# Concatenate the dataframes vertically using pd.concat().
defog_test = pd.concat(defog_test_list, axis = 0)

# Show the concatenated dataframe.
defog_test

In [ ]:
test = pd.concat([tdcsfog_test, defog_test], axis = 0).reset_index(drop = True)
test

In [ ]:
# Separate the dataset for the independent variables.
test_X = test.iloc[:, 1:4]

test_X

In [ ]:
# Get the predictions for the three models on the test data.
pred_y1 = gmodel1.predict(test_X)
pred_y2 = gmodel2.predict(test_X)
pred_y3 = gmodel3.predict(test_X)

test['StartHesitation'] = pred_y1 # target variable for StartHesitation
test['Turn'] = pred_y2 # target variable for Turn
test['Walking'] = pred_y3 # target variable for Walking

test

In [ ]:
submission = test.iloc[:, 4:].fillna(0.0)
submission

In [ ]:
submission.to_csv("submission.csv", index = False)